# Reasoning and Inference

## Overview

Build knowledge graphs, define rules, perform forward/backward chaining, and generate explanations for AI reasoning.

## Workflow: Build KG → Define Rules → Forward/Backward Chaining → Generate Explanations


In [ ]:
from semantica.kg import GraphBuilder


## Step 1: Build Knowledge Graph


In [ ]:
builder = GraphBuilder()

entities = [
    {"id": "alice", "type": "Person", "name": "Alice"},
    {"id": "bob", "type": "Person", "name": "Bob"},
    {"id": "charlie", "type": "Person", "name": "Charlie"},
    {"id": "sf", "type": "Location", "name": "San Francisco"},
    {"id": "california", "type": "Location", "name": "California"},
]

relationships = [
    {"source": "alice", "target": "bob", "type": "parent_of"},
    {"source": "bob", "target": "charlie", "type": "parent_of"},
    {"source": "sf", "target": "california", "type": "located_in"},
    {"source": "alice", "target": "sf", "type": "lives_in"},
]

knowledge_graph = builder.build(entities, relationships)


## Step 2: Define Rules


In [ ]:
class RuleManager:
    def __init__(self):
        self.rules = []
    
    def add_rules(self, rules):
        self.rules.extend(rules)

rule_manager = RuleManager()

rules = [
    "IF A is parent_of B AND B is parent_of C THEN A is grandparent_of C",
    "IF X is located_in Y AND Y is part_of Z THEN X is located_in Z",
    "IF X lives_in Y AND Y is located_in Z THEN X lives_in Z"
]

rule_manager.add_rules(rules)


## Step 3: Forward Chaining


In [ ]:
class InferenceEngine:
    def forward_chain(self, kg, rule_manager):
        new_facts = []
        
        for rule in rule_manager.rules:
            if "grandparent_of" in rule:
                parents = [r for r in relationships if r["type"] == "parent_of"]
                for p1 in parents:
                    for p2 in parents:
                        if p1["target"] == p2["source"]:
                            new_fact = {
                                "source": p1["source"],
                                "target": p2["target"],
                                "type": "grandparent_of",
                                "inferred": True
                            }
                            if new_fact not in new_facts:
                                new_facts.append(new_fact)
            
            elif "lives_in" in rule and "located_in" in rule:
                lives_in = [r for r in relationships if r["type"] == "lives_in"]
                located_in = [r for r in relationships if r["type"] == "located_in"]
                
                for live in lives_in:
                    for loc in located_in:
                        if live["target"] == loc["source"]:
                            new_fact = {
                                "source": live["source"],
                                "target": loc["target"],
                                "type": "lives_in",
                                "inferred": True
                            }
                            if new_fact not in new_facts:
                                new_facts.append(new_fact)
        
        return new_facts

inference_engine = InferenceEngine()
new_facts = inference_engine.forward_chain(knowledge_graph, rule_manager)

for fact in new_facts:
    print(f"{fact['source']} {fact['type']} {fact['target']} (inferred)")


## Step 4: Backward Chaining


In [ ]:
def backward_chain(kg, rule_manager, goal):
    proof_steps = []
    
    goal_source, goal_type, goal_target = goal
    
    for rel in relationships:
        if rel["source"] == goal_source and rel["type"] == goal_type and rel["target"] == goal_target:
            proof_steps.append({
                "step": "Direct fact",
                "fact": f"{goal_source} {goal_type} {goal_target}",
                "source": "knowledge_graph"
            })
            return proof_steps
    
    if goal_type == "grandparent_of":
        for rel1 in relationships:
            if rel1["source"] == goal_source and rel1["type"] == "parent_of":
                intermediate = rel1["target"]
                for rel2 in relationships:
                    if rel2["source"] == intermediate and rel2["type"] == "parent_of" and rel2["target"] == goal_target:
                        proof_steps.append({
                            "step": "Rule application",
                            "fact": f"{goal_source} parent_of {intermediate}",
                            "source": "knowledge_graph"
                        })
                        proof_steps.append({
                            "step": "Rule application",
                            "fact": f"{intermediate} parent_of {goal_target}",
                            "source": "knowledge_graph"
                        })
                        proof_steps.append({
                            "step": "Inference",
                            "fact": f"{goal_source} grandparent_of {goal_target}",
                            "source": "inference_rule"
                        })
                        return proof_steps
    
    return proof_steps

goal = ("alice", "grandparent_of", "charlie")
proof = backward_chain(knowledge_graph, rule_manager, goal)

for i, step in enumerate(proof, 1):
    print(f"Step {i}: {step['step']} - {step['fact']}")


## Step 5: Generate Explanations


In [ ]:
class ExplanationGenerator:
    def generate(self, proof, kg):
        if not proof:
            return "No proof found for the given goal."
        
        explanation_parts = []
        explanation_parts.append("Explanation:")
        
        for i, step in enumerate(proof, 1):
            if step['step'] == 'Direct fact':
                explanation_parts.append(f"{i}. We know that {step['fact']} from the knowledge graph.")
            elif step['step'] == 'Rule application':
                explanation_parts.append(f"{i}. From the knowledge graph: {step['fact']}.")
            elif step['step'] == 'Inference':
                explanation_parts.append(f"{i}. Therefore, by applying the inference rule: {step['fact']}.")
        
        return "\n".join(explanation_parts)

explanation_gen = ExplanationGenerator()
explanation = explanation_gen.generate(proof, knowledge_graph)
print(explanation)


## Summary

Reasoning and inference workflow:
- Knowledge Graph Built
- Inference Rules Defined
- Forward Chaining Performed
- Backward Chaining Performed
- Explanations Generated


In [ ]:
print("Reasoning and Inference Complete")
